In [244]:
import sqlite3
import pandas as pd
import sqlalchemy

import matplotlib.pyplot as plt
import numpy as np

from bs4 import BeautifulSoup 

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [245]:
# Load dataset
engine = sqlalchemy.create_engine('sqlite:///My_DB.db')
df = pd.read_sql('SELECT * FROM recipes', engine)
df.head()

,id,title,link,description,ingredients,photo
0,1,Жаркое по-деревенски,/recipes/recipe.php?rid=159506,"Простое, но очень ароматное и вкусное блюдо дл...","Продукты ,Свиная корейка без кости - 1 кг,Сало...",//img1.russianfood.com/dycontent/images_upl/46...
1,2,Куриный беф-строганов,/recipes/recipe.php?rid=121844,Бефстроганов из курицы - очень нежный и вкусны...,"Продукты ,(на 4 порции),Куриное филе – 600 г,С...",//img1.russianfood.com/dycontent/images_upl/32...
2,3,"Картошка, запечённая с сосисками, помидорами и...",/recipes/recipe.php?rid=165007,"Простое, но достаточно интересное блюдо - запе...","Продукты ,(на 4 порции),Картофель (некрупный) ...",//img1.russianfood.com/dycontent/images_upl/55...
3,4,"Макароны с мясом в томатном соусе, на сковороде",/recipes/recipe.php?rid=164739,"Блюдо привлекательно тем, что макароны отдельн...","Продукты ,Свинина (корейка без кости) - 300 г,...",//img1.russianfood.com/dycontent/images_upl/55...
4,5,Жюльен с курицей и грибами,/recipes/recipe.php?rid=140728,Очень популярную закуску - жульен (или точнее ...,"Продукты ,(на 4 порции),Филе куриное - 400 г,Г...",//img1.russianfood.com/dycontent/images_upl/19...


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           65 non-null     int64 
 1   title        65 non-null     object
 2   link         65 non-null     object
 3   description  65 non-null     object
 4   ingredients  65 non-null     object
 5   photo        65 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.2+ KB


# EDA

In [247]:
# Check the NA's
df.isnull().sum()

id             0
title          0
link           0
description    0
ingredients    0
photo          0
dtype: int64

## Creating corpus

In [248]:
# Combining title, description and ingredients columns
df['combined_text'] = df['title']+ ' ' + df['description']+ ' ' + df['ingredients']+ ' '

In [249]:
df['combined_text'][0]

'Жаркое по-деревенски Простое, но очень ароматное и вкусное блюдо для семейного обеда или ужина. Продукты ,Свиная корейка без кости - 1 кг,Сало свиное свежее - 50 г,Масло сливочное - 50 г,Картофель - 1 кг,Перец болгарский - 1 шт.,Морковь - 2 шт.,Лук репчатый - 2 шт.,Чеснок - 2 зубчика,Зелень петрушки свежая - по вкусу,Зелень петрушки сушёная - по вкусу,Специи для мяса - по вкусу,Перец чёрный горошком - по вкусу,Перец чёрный молотый - по вкусу,Соль - по вкусу '

In [250]:

from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('russian')
stop_words_ = set(stopwords.words('russian'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)
    

In [251]:
tokenizer = RegexpTokenizer(r'\w+')

In [252]:
def clean_txt(text):
    combined_tokens = tokenizer.tokenize(text.lower())
    no_stop_words = [token for token in combined_tokens if token not in stopwords.words('russian') and len(token)>2]
    without_gram = [token for token in no_stop_words if token not in ['ч','кг','г','шт', 'ст', 'ложки', 'стакан','мл']]
    without_gram = list(dict.fromkeys(without_gram ))
    return " ".join(without_gram)

In [253]:
df['combined_text'] = df['combined_text'].apply(clean_txt)

In [254]:
df['combined_text'][5]

'ленивые голубцы духовке хочется голубцов времени приготовление выручает ленивый вариант приготовления блюда продукты фарш мясной 400 капуста белокочанная рис круглозёрный стакана яйцо лук репчатый морковь чеснок зубчика масло подсолнечное ложка перец чёрный молотый вкусу соль соуса томатная паста воды томатный сок подачи сметана зелень свежая'

#### TF-IDF ( Term Frequency - Inverse Document Frequency ) 

In [255]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_recipe = tfidf_vectorizer.fit_transform((df['combined_text'])) #fitting and transforming the vector
tfidf_recipe

<65x1266 sparse matrix of type '<class 'numpy.float64'>'
	with 3054 stored elements in Compressed Sparse Row format>

In [256]:
promt_request = ['Жаркое жюльен макароны помидоры мясо сыр суши']

In [257]:
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(promt_request)
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x), tfidf_recipe)

In [258]:
output = list(cos_similarity_tfidf)

In [259]:
def get_recommendation(top, df, scores):
    recommendation = pd.DataFrame(columns = ['request', 'number',  'title', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'request'] = promt_request
        recommendation.at[count, 'title'] = df['combined_text'][i]
        recommendation.at[count, 'number'] = df['id'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [260]:
top = sorted(range(len(output)), key=lambda i: output[i], reverse=True)[:10]
list_scores = [output[i][0][0] for i in top]
get_recommendation(top,df, list_scores)

,request,number,title,score
0,[Жаркое жюльен макароны помидоры мясо сыр суши],5,жюльен курицей грибами очень популярную закуск...,0.132864
1,[Жаркое жюльен макароны помидоры мясо сыр суши],33,жаркое свинины картошкой выходной день обед пр...,0.129378
2,[Жаркое жюльен макароны помидоры мясо сыр суши],24,запеканка макарон курицей помидорами доступных...,0.098856
3,[Жаркое жюльен макароны помидоры мясо сыр суши],1,жаркое деревенски простое очень ароматное вкус...,0.096738
4,[Жаркое жюльен макароны помидоры мясо сыр суши],27,гнёзда спагетти фаршем помидорами сыром духовк...,0.08772
5,[Жаркое жюльен макароны помидоры мясо сыр суши],46,макароны крабовыми палочками сметаной чесноком...,0.081135
6,[Жаркое жюльен макароны помидоры мясо сыр суши],19,басма величество задача накормить человек напр...,0.079197
7,[Жаркое жюльен макароны помидоры мясо сыр суши],9,спаггети курицей помидорами черри простой вкус...,0.076778
8,[Жаркое жюльен макароны помидоры мясо сыр суши],61,мясо немецки белым вином помидорами яблоками и...,0.074056
9,[Жаркое жюльен макароны помидоры мясо сыр суши],8,тушёная картошка мясом грибами сметаной картоф...,0.065676


In [261]:
df['combined_text'][4]

'жюльен курицей грибами очень популярную закуску жульен точнее приготовить сковороде примерно полчаса совершенно напрягая приобретением кокотниц доведением блюда готовности духовке получается вкусно продукты порции филе куриное 400 грибы шампиньоны 300 лук репчатый 100 сливки 200 сыр твердый 150 масло растительное соль вкусу перец черный молотый зелень базилик салатные листья украшения'